In [70]:
#讀入向量空間模型，給定任一篇文件，以訓練學習演算法自動判斷類別
#Big Data & Business Analytics, Willie Yang, National Taiwan University

In [1]:
from scipy import sparse
X = sparse.load_npz("limit_model.npz") #讀入稀疏矩陣
X

<7393x108 sparse matrix of type '<class 'numpy.float64'>'
	with 173482 stored elements in Compressed Sparse Row format>

In [2]:
#以下套用之前範例讀入詞彙集
import csv

termset=dict() #用字典預備儲存所有詞彙
no=0 #幫詞彙編號

f = open('limit_up_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

f = open('limit_down_term.csv', newline='', encoding='utf-8') #開檔 
rows = csv.reader(f, delimiter=',') #讀檔
for row in rows:
  if row[0] not in termset: #如果是新詞彙則加入(意思是重複的不列入)
    termset[row[0]]=no
    no+=1
f.close()

print("done")

done


In [3]:
#給定任何一段內容當作查詢
query='PC品牌大廠華碩（2357）今（16）日舉行線上法說會，受惠顯示卡及PC需求暢旺，\
  帶動華碩去年品牌毛利率、營益率、淨利率「三率三升」，稅後純益大增68%，創下歷史新高，\
  全年每股純益（EPS）達60元，大賺近6個股本。華碩今日公布去年品牌財報，毛利率20.3%，\
  年增3.1個百分點。營益率9.6%，年增3個百分點。稅後純益445.5億元，年增68%。\
  淨利率8.9%，年增2個百分點。全年每股純益達60元'
query='台股加權指數在最近9個交易日，從最高到最低點，跌了2,544點，創下史上最快速的失速列車紀錄；\
  12日台股盤中急挫1,418點，市場衰鴻遍野，據統計，盤中最多曾有711檔個股觸及跌停、占上市櫃的四成比重，\
  最後仍有251檔收跌停，其中，陽明等15檔股價亮燈跌停，仍有7千張以上賣單高掛，貨櫃三雄均入榜。'

#以下套用斷詞小範例
import monpa
from monpa import utils
str='' #暫存本篇斷詞後的內容用
sentence_list = utils.short_sentence(query) #斷句
for item in sentence_list:
  result_cut = monpa.cut(item) #斷詞
  for term in result_cut:
    term=term.strip() #去除前後多餘空白
    if(len(term)>1): #若詞長>1
      str=str+' '+term
print(str)

+---------------------------------------------------------------------+
  Welcome to MONPA: Multi-Objective NER POS Annotator for Chinese
+---------------------------------------------------------------------+
已找到 model檔。Found model file.
 台股 加權 指數 最近 交易日 低點 2,544 創下 快速 失速 列車 紀錄 12 台股 急挫 1,418 市場 衰鴻 遍野 統計 最多 711 個股 觸及 跌停 市櫃 四成 比重 最後 251 跌停 其中 陽明 15 股價 亮燈 跌停 7千 以上 賣單 高掛 貨櫃 入榜


In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(vocabulary=termset,use_idf=True) #用給定的termset建構向量，並使用idf加權 (這邊作法值得商榷)
q=vectorizer.fit_transform([str]) #將查詢也套入同一個向量空間
q

/Users/andy/opt/anaconda3/envs/monpa-env/lib/python3.10/site-packages/sklearn/feature_extraction/text.py:1380: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


<1x108 sparse matrix of type '<class 'numpy.float64'>'
	with 5 stored elements in Compressed Sparse Row format>

In [10]:
print(q) #印出該查詢做觀察

  (0, 51)	0.35355339059327373
  (0, 8)	0.35355339059327373
  (0, 5)	0.35355339059327373
  (0, 2)	0.35355339059327373
  (0, 1)	0.7071067811865475


In [8]:
"""import pandas as pd
pd.DataFrame(X.toarray(),columns=termset) #將X印出做觀察"""
# 這個資料太多打開會爆掉

'import pandas as pd\npd.DataFrame(X.toarray(),columns=termset) #將X印出做觀察'

In [9]:
# 用這個看前十筆
for i in range(10):
    print(f"\n--- 第 {i+1} 篇文章 ---")
    print("出現過的詞索引：", X[i].nonzero()[1])
    print("TF-IDF 分數：", X[i].data)


--- 第 1 篇文章 ---
出現過的詞索引： [104  99  93  69  67  65  60  57  56  55  54  52  44  33  27  23  13   2
   1   0]
TF-IDF 分數： [0.17990612 0.15917779 0.17197084 0.31819512 0.37082592 0.17934805
 0.16354253 0.1507458  0.13385521 0.33155738 0.16150906 0.16537817
 0.12689383 0.15339457 0.30382613 0.14014206 0.29107821 0.34496908
 0.20061711 0.12443468]

--- 第 2 篇文章 ---
出現過的詞索引： [104  96  93  76  74  68  61  58  50  46  36  25  21  19  10   8   7   5
   2   1   0]
TF-IDF 分數： [0.16134262 0.32318912 0.15422613 0.15075331 0.1565294  0.14552073
 0.13970617 0.15167531 0.14178954 0.14133148 0.12120446 0.12642828
 0.13336731 0.13733755 0.41597869 0.10142001 0.10818752 0.11292993
 0.10312455 0.62970795 0.11159496]

--- 第 3 篇文章 ---
出現過的詞索引： [104  99  97  94  93  91  88  84  79  74  66  63  62  60  57  56  55  54
  51  48  44  40  36  33  30  25  24  23  22  20  19  12  10   9   8   5
   4   2   1   0]
TF-IDF 分數： [0.07951418 0.07035276 0.08260572 0.17788375 0.07600698 0.07642439
 0.07818423 0.07251123 0.14

In [11]:
y=[] #用以儲存已知類別
for i in range(4194): #已知前4194篇為漲停，標記為看漲 (此為取巧權宜作法)
  y.append(['看漲'])
for i in range(3199): #已知後3199篇為跌停，標記為看跌
  y.append(['看跌'])

In [16]:
"""import pandas as pd
pd.DataFrame(y) #將y印出做觀察"""
# 這個也會爆開，看十筆就好

for i in range(4190, 4200):
    print(f"第 {i+1} 筆標籤：", y[i])

第 4191 筆標籤： ['看漲']
第 4192 筆標籤： ['看漲']
第 4193 筆標籤： ['看漲']
第 4194 筆標籤： ['看漲']
第 4195 筆標籤： ['看跌']
第 4196 筆標籤： ['看跌']
第 4197 筆標籤： ['看跌']
第 4198 筆標籤： ['看跌']
第 4199 筆標籤： ['看跌']
第 4200 筆標籤： ['看跌']


In [17]:
import pandas as pd

pd.DataFrame(q.toarray(),columns=termset) #將q印出做觀察

ImportError: numpy.core.multiarray failed to import (auto-generated because you didn't call 'numpy.import_array()' after cimporting numpy; use '<void>numpy._import_array' to disable if you are certain you don't need it).

In [18]:
from sklearn.naive_bayes import MultinomialNB
classifier = MultinomialNB() #naive bayes classifier
classifier.fit(X, y) #訓練

/Users/andy/opt/anaconda3/envs/monpa-env/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [19]:
y_pred = classifier.predict(q) #預測
y_pred

array(['看漲'], dtype='<U2')

In [20]:
#以下將自身資料切成train及test兩組，重新訓練一次，測試模型準確率
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30) #隨機挑選30%當測試資料
classifier = MultinomialNB()
classifier.fit(X_train, y_train) #訓練

/Users/andy/opt/anaconda3/envs/monpa-env/lib/python3.10/site-packages/sklearn/utils/validation.py:1143: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


MultinomialNB()

In [21]:
from sklearn import metrics 
from sklearn.metrics import accuracy_score
y_pred= classifier.predict(X_test) #用測試資料預測
print("Accuracy:",metrics.accuracy_score(y_test, y_pred)) #比對答案，計算準確率

Accuracy: 0.5563570784490532


In [22]:
from sklearn.metrics import classification_report  
print(classification_report(y_test, y_pred)) #印出分類報告

              precision    recall  f1-score   support

          看漲       0.57      0.89      0.70      1255
          看跌       0.46      0.12      0.19       963

    accuracy                           0.56      2218
   macro avg       0.51      0.51      0.44      2218
weighted avg       0.52      0.56      0.47      2218



In [23]:
from sklearn.metrics import confusion_matrix  
print(confusion_matrix(y_test, y_pred, labels=['看漲','看跌'])) #印出混淆矩陣
# test/predicted 看漲   看跌
#    看漲         TN     FP
#    看跌         FN     TP

[[1122  133]
 [ 851  112]]
